In [1]:
import pickle
import operator
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt 
from cv2 import cv2
import cv2
import math

import keras
from keras import *
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input, decode_predictions, VGG16
from keras.preprocessing import image as kimage
from keras.models import Sequential
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.optimizers import SGD
from keras.layers import Activation, LeakyReLU


import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KDTree
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, LeaveOneOut
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

import pickle

import tensorflow as tf
from tensorflow.keras.activations import sigmoid
from tensorflow.keras import regularizers
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import efficientnet
import efficientnet.keras as efn


from skimage.feature import local_binary_pattern
from skimage.feature import hog

from PIL import Image


Si definisce una funzione per plottare accuracy e loss attraverso le epoche

In [ ]:
x_plot = list(range(1,30+1))

def plot_history(network_history):
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(x_plot, network_history.history['loss'])
    plt.plot(x_plot, network_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(x_plot, network_history.history['accuracy'])
    plt.plot(x_plot, network_history.history['val_accuracy'])
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()


# Neural Network

Vengono creati train, test e validation generator, con un preprocessing sulle immagini uguale a quello attuato dalle reti pre-addestrate che si vogliono utilizzare sulle immagini di Imagenet.

In [ ]:
train_processing = keras.preprocessing.image.ImageDataGenerator(rotation_range=45, 
                                                                width_shift_range=0.1, 
                                                                height_shift_range=0.1, 
                                                                zoom_range=0.1, 
                                                                preprocessing_function=efficientnet.preprocess_input) ### mobilenet_v2, efficientnet
train_generator= train_processing.flow_from_directory( 
    directory='mask_NEW_train_folders',
    target_size=(224,224),
    color_mode= 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 1
)

In [ ]:
test_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=efficientnet.preprocess_input) ### mobilenet_v2, efficientnet
test_generator= test_processing.flow_from_directory(
    directory='mask_NEW_test_folders',
    target_size=(224,224),
    color_mode= 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 1
)

In [ ]:
val_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=efficientnet.preprocess_input) ### mobilenet_v2, efficientnet
val_generator= val_processing.flow_from_directory(
    directory='mask_NEW_val_folders',
    target_size=(224,224),
    color_mode= 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 1
)

In [ ]:
base_net = efficientnet.EfficientNetB0(input_shape=(224,224,3), weights ='imagenet', include_top=False, pooling='avg') ## mobilenet_v2.MobileNetV2, efficientnet.EfficientNetB0, efficientnet.EfficientNetB2

In [ ]:
base_net.summary()

## Rete iniziale

In [ ]:
for layers in base_net.layers:
    layers.trainable = False

x = base_net.output 

x = keras.layers.Dropout(0.5)(x)

pred = keras.layers.Dense(102, activation='softmax')(x) 

In [ ]:
net = keras.Model(inputs=base_net.input, outputs=pred)

In [ ]:
net.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=keras.optimizers.Adam(), 
            metrics=['accuracy'])

In [ ]:
history = net.fit(train_generator, batch_size=64, epochs=30, verbose=1, validation_data=val_generator)

In [ ]:
plot_history(history)

In [ ]:
pred_test = net.evaluate(test_generator)
print(pred_test)
pred_val = net.evaluate(val_generator)
print(pred_val)

In [ ]:
net.save('effB0_NoRelu_Adam.h5')

si calcola e si plotta la confusion matrix

In [ ]:
test_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function= efficientnet.preprocess_input)###resnet_v2 mobilenet_v2 vgg16
test_generator= test_processing.flow_from_directory(
    directory='mask_NEW_test_folders',
    target_size=(224,224), #scelta standard
    color_mode= 'rgb', #dobbiamo mettere stessi parametri della rete di partenza
    class_mode = 'categorical',
    batch_size = 1020,
    shuffle = True,
    seed = 1
)

In [ ]:
X_test, yc_test = next(test_generator)
y_test = np.argmax(yc_test, axis=1)

In [ ]:
y_pred = model.predict(X_test,  verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

In [ ]:
cm=confusion_matrix(y_test, y_pred_bool)
plt.imshow(cm, cmap=plt.cm.Blues)

## Rete completa

In [ ]:
for layers in base_net.layers:
    layers.trainable = False
    
x = base_net.output 

x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(128, activation = 'relu', kernel_regularizer=regularizers.l2(0.0005))(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)

pred = keras.layers.Dense(102, activation='softmax')(x) 


In [ ]:
net = keras.Model(inputs=base_net.input, outputs=pred)

In [ ]:
net.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=keras.optimizers.Adam(), 
            metrics=['accuracy'])

In [ ]:
history = net.fit(train_generator, batch_size=64, epochs=30, verbose=1, validation_data=val_generator)

In [ ]:
plot_history(history)

In [ ]:
pred_test = net.evaluate(test_generator)
print(pred_test)
pred_val = net.evaluate(val_generator)
print(pred_val)

In [ ]:
net.save('effB0_Relu_Adam.h5')

In [ ]:
test_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function= efficientnet.preprocess_input)###resnet_v2 mobilenet_v2 vgg16
test_generator= test_processing.flow_from_directory(
    directory='mask_NEW_test_folders',
    target_size=(224,224), #scelta standard
    color_mode= 'rgb', #dobbiamo mettere stessi parametri della rete di partenza
    class_mode = 'categorical',
    batch_size = 1020,
    shuffle = True,
    seed = 1
)

In [ ]:
X_test, yc_test = next(test_generator)
y_test = np.argmax(yc_test, axis=1)

In [ ]:
y_pred = model.predict(X_test,  verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

In [ ]:
cm=confusion_matrix(y_test, y_pred_bool)
plt.imshow(cm, cmap=plt.cm.Blues)

# SVM

Si importa la rete pre-addestrata scelta

In [3]:
net = efficientnet.EfficientNetB0(include_top=False, weights='imagenet', pooling='avg', input_shape=(224,224,3)) ## mobilenet_v2.MobileNetV2, efficientnet.EfficientNetB0, efficientnet.EfficientNetB2

neural features: utilizza la rete scelta, per classificare l'immagine fornita in input, dopo aver applicato lo stesso preprocessing applicato dalla rete alle immagini di Imagenet

get_colors: dopo avere approssimato ogni colore dell'immagine fornita al colore RGB limite più vicino, calcola la percentuale di blu, rosso e verde contenuta nell'immagine

hog_image: calcola l'HoG dell'immagine utilizzando 8 direzioni

lbp_image_2: applica un LBP di raggio 5, che considera 8 punti sul raggio

combo_xxx: combina le features indicate nel nome

load_data: carica i dati calcolando le features indicate in 'feature extractor'

In [ ]:
def neural_features(image_path):
    img = kimage.load_img(image_path, target_size=(224, 224))
    x = kimage.img_to_array(img)
    x = efficientnet.preprocess_input(x) #mobilenet_v2, efficientnet
    x = np.expand_dims(x, axis = 0)
    features = net.predict(x) #viene utilizzata la rete importata all'inizio del chunk
    features = features.flatten()   
    return features

def get_colors (image_path):
    img = cv2.imread(image_path)
    _, img = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)

    color = ('b','g','r')
    qtdBlue = 0
    qtdGreen = 0
    qtdRed = 0
    totalPixels = 0

    for channel,_ in enumerate(color):
        histr = cv2.calcHist([img],[channel],None,[256],[1,256])
        totalPixels+=sum(histr)
        if channel==0:
            qtdBlue = sum(histr)
        elif channel==1:
            qtdGreen = sum(histr)
        elif channel==2:
            qtdRed = sum(histr)

    qtdBlue = (qtdBlue/totalPixels)*100
    qtdGreen = (qtdGreen/totalPixels)*100
    qtdRed = (qtdRed/totalPixels)*100

    if math.isnan(qtdBlue) == True :
        qtdBlue = np.zeros(1)
    if math.isnan(qtdGreen) == True :
        qtdGreen = np.zeros(1)
    if math.isnan(qtdRed) == True :
        qtdRed = np.zeros(1)

    colors = np.asarray([qtdBlue[0], qtdGreen[0], qtdRed[0]])
    return colors

def hog_image (image_path):
    image = kimage.load_img(image_path, target_size= (224,224))
    features, _ = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True, multichannel=True)

    return features

def lbp_image_2 (image_path):

    img_bgr = cv2.imread(image_path)
    img_bgr = cv2.resize(img_bgr, (100,100)) 
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY) 

    lbp = local_binary_pattern(img_gray, 8,5 , "uniform")

    lbp_array_flat = lbp_array.flatten(lbp)

    return lbp_array_flat

def combo_neural_colors_hog(image_path):

    neural = neural_features (image_path)

    colors = get_colors(image_path)
    
    hog = hog_image (image_path)
    
    return np.concatenate((colors, neural, hog))

def combo_neural_colors_hog_lbp(image_path):

    neural = neural_features (image_path)

    colors = get_colors(image_path)
    
    hog = hog_image (image_path)
    
    lbp = lbp_image_2 (image_path)
    
    return np.concatenate((colors, neural, hog, lbp))

def combo_colors_hog_lbp(image_path):

    lbp = lbp_image_2 (image_path)

    colors = get_colors(image_path)
    
    hog = hog_image (image_path)
    
    return np.concatenate((colors, lbp, hog))

In [5]:
def identity(image_path):
    img = kimage.load_img(image_path, target_size=(224, 224), color_mode= 'rgb')
    x = kimage.img_to_array(img)
    x = efficientnet.preprocess_input(x)
    return x


def load_data(folder, feature_extractor=identity, maximages = 'default' ):
    if maximages == 'default':
        maximages = len(os.listdir(folder))+1

    base_path = folder

    features = []

    for index, image in enumerate(sorted(os.listdir(base_path))):
        print ('image', index)
        if index < maximages:
            image_path = base_path+image
            # Load file and extract features
            cur_features = feature_extractor(image_path)
            #cur_features = cur_features.flatten()
            features.append(cur_features)

    #features = np.array(features)
    return features

Inizialmente si crea un dataset con le features estratte dal 'feature_extractor' scelto

In [ ]:
X_train_features = load_data('mask_NEW_train/',feature_extractor=combo_neural_colors_hog)
with open('X_train_features_neural_colors_hog.pickle', 'wb') as handle:
    pickle.dump(X_train_features, handle)
X_test_features = load_data(feature_extractor=combo_neural_colors_hog, folder = 'mask_NEW_test/')
with open('X_test_features_neural_colors_hog.pickle', 'wb') as handle:
    pickle.dump(X_test_features, handle)
X_val_features = load_data(feature_extractor=combo_neural_colors_hog, folder = 'mask_NEW_val/')
with open('X_val_features_neural_colors_hog.pickle', 'wb') as handle:
    pickle.dump(X_val_features, handle)

In [ ]:
with open ('102flowers_y_train.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
num_classes = 102
yc_train = keras.utils.to_categorical(y_train-1, num_classes)
with open ('102flowers_y_test.pickle', 'rb') as handle:
    y_test = pickle.load(handle)
num_classes = 102
yc_test = keras.utils.to_categorical(y_test-1, num_classes)
with open ('102flowers_y_val.pickle', 'rb') as handle:
    y_val = pickle.load(handle)
num_classes = 102
yc_val = keras.utils.to_categorical(y_val-1, num_classes)
with open ('102flowers_y_test_val.pickle', 'rb') as handle:
    y_test_val = pickle.load(handle)
num_classes = 102
yc_test_val = keras.utils.to_categorical(y_test_val-1, num_classes)

In [ ]:
with open('X_test_features_neural_colors_hog.pickle', 'rb') as handle:
    X_test_features = pickle.load(handle)
with open('X_train_features_neural_colors_hog.pickle', 'rb') as handle:
    X_train_features = pickle.load(handle)
with open('X_val_features_neural_colors_hog.pickle', 'rb') as handle:
    X_val_features = pickle.load(handle)
with open('X_test_val_features_neural_colors_hog.pickle', 'rb') as handle:
    X_test_val_features = pickle.load(handle)

si impostano empricamente C e gamma tenendo conto che un C basso implica una forte regolarizzazione

In [ ]:
clf2 = SVC(kernel='poly',  degree=2, class_weight='balanced', C = 0.8, gamma = 0.005)
print('fit...')
clf2=clf2.fit(X_train_features, y_train)
print('predict...')
y2_pred = clf2.predict(X_train_features)

print(classification_report(y_train, y2_pred))

with open('svc_neural_efficient_C08G0005.pickle', 'wb') as handle:
    pickle.dump(clf2, handle)

In [ ]:
y_pred_test = clf2.predict(X_test_features)
print(classification_report(y_test, y_pred_test))
y_pred_val = clf2.predict(X_val_features)
print(classification_report(y_val, y_pred_val))

In [ ]:
cm=confusion_matrix(y_test, y_pred_test)
plt.imshow(cm, cmap=plt.cm.Blues)